In [22]:

class NimState():

    def __init__(self, p, n, k):
        self.current_player = p  # 1 for Player 1, 2 for Player 2
        self.n = n
        self.k = k

    def is_terminal(self):
        return self.n <= 1

    def generate_child_states(self):
        child_states = []
        for stones in range(1, min([self.k, self.n]) + 1):
            remaining_stones = self.n - stones
            child_state = NimState(-self.current_player, remaining_stones, self.k)
            child_states.append(child_state)
        return child_states

    def get_next_player(self):
        return -self.current_player
    
    def get_winner(self):
        if self.is_terminal():
            return self.current_player
        else:
            return 0

In [23]:
from collections import deque
import copy

def print_matrix(matrix):
    for i in matrix:
        print(i)

class HexState():

	def __init__(self, board, player, move=None):
		self.board = board
		self.current_player = player
		self.move = copy.copy(move)
		self.move = move

	def check_full_board(self):
		"""
    	Checks if a 2D list is filled with no 0s and that the number of 1s and -1s are equal.
    	"""
		for row in self.board:
			for element in row:
				if element == 0:
					return False
		return True


	def get_valid_moves(self):
		valid_cells = []
		for r in range(len(self.board)):
			for c in range(len(self.board)):
				if self.board[r][c] == 0:
					valid_cells.append((r, c))
		return valid_cells
	

	def execute_move(self, move):
		self.move = copy.copy(move)
		r = move[0]
		c = move[1]
		if self.board[r][c] != 0:
			raise Exception('ILLEGAL MOVE!')
		self.board[r][c] = -self.current_player


	def generate_child_states(self):
		child_states = []
		valid_moves = self.get_valid_moves()
		for move in valid_moves:
			board_copy = copy.deepcopy(self.board)
			child_state = HexState(board_copy, -self.current_player)
			child_state.execute_move(move)
			#if move[0] == 0 and move[1] == 3 and self.gen == 1:
				#print(child_state.board, self.current_player)
			child_states.append(child_state)
		return child_states

	
	def get_next_player(self):
		return -self.current_player


	def get_winner(self):
		matrix = copy.deepcopy(self.board)
		player = -self.current_player #Fix rekkefølge
		queue = deque()
		visited = set()
		#if player == 1: kan prøve DS istedenfor
		for r in range(len(matrix[0])):
			if matrix[r][0] == 1:
				queue.append((r,0))
				visited.add((r,0))
		while queue:
			r, c = queue.popleft()
			if c == len(matrix) - 1:
				return 1
			for x, y in ((r-1,c), (r,c-1), (r+1,c), (r,c+1), (r-1,c+1), (r+1,c-1)):
				if 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] == 1 and (x,y) not in visited:
					queue.append((x,y))
					visited.add((x,y))
		for c in range(len(matrix[0])):
			if matrix[0][c] == -1:
				queue.append((0,c))
				visited.add((0,c))
		while queue:
			r, c = queue.popleft()
			if r == len(matrix) - 1:
				return -1
			for x, y in ((r-1,c), (r,c-1), (r+1,c), (r,c+1), (r-1,c+1), (r+1,c-1)):
				if 0 <= x < len(matrix) and 0 <= y < len(matrix[0]) and matrix[x][y] == -1 and (x,y) not in visited:
					queue.append((x,y))
					visited.add((x,y))
		return 0

In [24]:
"""
board = [[1, 1, -1, -1],
		 [1, -1, 1, -1],
		 [-1, 1, -1, -1],
		 [1, 1, -1, 1]]
"""
board = [[1, 1, 1, 1], 
 		 [1, -1, -1, 1], 
 		 [-1, 1, -1, -1], 
 		 [-1, -1, -1, -1]]
         
state = HexState(board, 1)
print(state.get_winner())

"""
s0 = HexState(board, 1)
s0.execute_move([0,1])
s0.execute_move([0,0])
print(s0.get_winner())
s0.execute_move([0,2])
print(s0.get_winner())

def print_matrix(matrix):
    for i in matrix:
        print(i)
        
print_matrix(s0.board)
"""

1


'\ns0 = HexState(board, 1)\ns0.execute_move([0,1])\ns0.execute_move([0,0])\nprint(s0.get_winner())\ns0.execute_move([0,2])\nprint(s0.get_winner())\n\ndef print_matrix(matrix):\n    for i in matrix:\n        print(i)\n        \nprint_matrix(s0.board)\n'

# Node class

In [25]:
import random

class Node:

    def __init__(self, state, parent=None, gen=0):
        self.parent = parent
        self.state = state
        self.children = []
        self.wins = 0
        self.visits = 0 
        self.gen = gen

    def get_children(self):
        children = []
        for state in self.state.generate_child_states():
            child = Node(state, parent=self, gen=self.gen+1)
            children.append(child)
        return children

    def get_random_child(self):
        if self.children:
            return random.choice(self.children)
        else:
            return random.choice(self.get_children())

    def expand(self):
        if not self.children:
            self.children = self.get_children()

# MCTS Class

In [26]:
import math
import random

class MCTS:
    
    def __init__(self, params):
        self.params = params

    def select_action(self, node, starting_player, is_random=False):
        self.update(node, self.params['num_simulations'], starting_player)  # Update the tree using mcts

        if is_random:
            return random.choice(node.children)

        current_player = node.state.current_player
        action_node = None

        highest_qsa = -float('inf')
        lowest_qsa = float('inf')

        for child in node.children:
            qsa = float(child.wins)/float(child.visits)  # Calculate Q(s,a)

            if starting_player == current_player:
                if qsa > highest_qsa:
                    highest_qsa = qsa
                    action_node = child
            else:  # If the current player is the opposing player, the best score is the lowest Q(s,a)
                if qsa < lowest_qsa:
                    lowest_qsa = qsa
                    action_node = child
        return action_node

    def update(self, node, num_simulations, current_player):
        for _ in range(num_simulations):
            best_node = self.tree_search(node, current_player)
            best_node.expand()
            if len(best_node.get_children()) > 0:  # Chose a random child if just expanded
                best_node = random.choice(best_node.children)
            winner = self.evaluate(best_node)
            self.backpropagate(best_node, winner, current_player)

    def tree_policy_value(self, parent, child, is_opponent):
        q_value = child.wins / (child.visits + 1)
        u_value = self.params['C'] * math.sqrt(math.log(parent.visits) / (child.visits + 1))

        if is_opponent:
            return q_value - u_value
        return q_value + u_value

    def tree_search(self, node, current_player):
        if not node.children:  # Breaks recursion and returns the best leaf node
            return node

        best_child = node
        highest_value = -float('inf')
        lowest_value = float('inf')
        opposing_player = node.state.current_player != current_player

        for child in node.children:
            value = self.tree_policy_value(node, child, opposing_player)  # Get value of node based on the tree policy

            if opposing_player and value < lowest_value:
                # The best value is the lowest value when the player is the opposing player
                best_child = child
                lowest_value = value

            elif (not opposing_player) and value > highest_value:
                best_child = child
                highest_value = value
        return self.tree_search(best_child, current_player)  # Recursively search the tree until reaching best leaf node

    @staticmethod
    def evaluate(node):
        winner = node.state.get_winner()
        while winner == 0: #Kan optimaliseres
            node = node.get_random_child()
            winner = node.state.get_winner()

        #winner = node.state.get_winner()  #Før: winner = node.state.get_next_player()
        return winner

    @staticmethod
    def backpropagate(node, winner, current_player):
        while node is not None:
            if winner == current_player:
                node.wins += 1
            node.visits += 1
            node = node.parent

# Main

In [27]:
"""
N = 5
K = 3
root_node = Node(NimState(1, N, K))

params = {
	'num_simulations': 100000,
	'C': 1.4
}

mcts = MCTS(params)
action_node = mcts.select_action(root_node, starting_player=1)
for child in root_node.children:
	print(f'Stones removed: {root_node.state.n - child.state.n}, Visits: {child.visits}, Wins: {child.wins}')
"""

"\nN = 5\nK = 3\nroot_node = Node(NimState(1, N, K))\n\nparams = {\n\t'num_simulations': 100000,\n\t'C': 1.4\n}\n\nmcts = MCTS(params)\naction_node = mcts.select_action(root_node, starting_player=1)\nfor child in root_node.children:\n\tprint(f'Stones removed: {root_node.state.n - child.state.n}, Visits: {child.visits}, Wins: {child.wins}')\n"

In [28]:
"""
board = [[0, 0, -1, 0],
         [1, 0,  1, 0],
         [0, 1, -1, 0],
		 [0, 0, -1, 1]] 
         
board = [[1, 1, 1, 0],
         [0, 0, -1, 0],
         [-1, 0, -1, 0],
		 [0, 0, 0, 0]] 
"""

board = [[1, 0, 1, 0],
	 	 [0, 0, -1, 0],
		 [0, 0, -1, 0],
		 [0, 0, -1, 0]]


state = HexState(board, -1)

root = Node(state)

params = {
	'num_simulations': 1000,
	'C': 1.4
}

mcts = MCTS(params)
action_node = mcts.select_action(root, starting_player=-1)
print(f'Action: {action_node.state.move}')
for child in root.children:
	print(f'Move: {child.state.move}, Visits: {child.visits}, Wins: {child.wins}')

Action: (0, 3)
Move: (0, 1), Visits: 116, Wins: 98
Move: (0, 3), Visits: 388, Wins: 388
Move: (1, 0), Visits: 72, Wins: 55
Move: (1, 1), Visits: 85, Wins: 67
Move: (1, 3), Visits: 56, Wins: 39
Move: (2, 0), Visits: 45, Wins: 29
Move: (2, 1), Visits: 45, Wins: 29
Move: (2, 3), Visits: 59, Wins: 42
Move: (3, 0), Visits: 61, Wins: 44
Move: (3, 1), Visits: 38, Wins: 23
Move: (3, 3), Visits: 35, Wins: 20


# Test the MCTS-agent

In [29]:
def print_board(board):
    for row in board:
        print(row)
    print()

In [35]:
board = [[0, 0, 0, 0, 0, 0, 0],
	 	 [0, 0, 0, 0, 0, 0, 0],
		 [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
		 [0, 0, 0, 0, 0, 0, 0]]

params = {
	'num_simulations': 100,
	'C': 1.4
}

results = []

for _ in range(10):
    
    mcts = MCTS(params)

    s0 = HexState(board, 1)
    node = Node(s0)

    while node.state.get_winner() == 0:
        if node.state.current_player == 1:
            node = mcts.select_action(node, node.state.current_player)
        elif node.state.current_player == -1:
            node = mcts.select_action(node, node.state.current_player, is_random=True)
            
    results.append(node.state.get_winner())
        
print(results)

[-1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# Husk å sjekk generasjonen til noden/antall visits i sluttnode

In [34]:
for child in node.children:
	print(f'Move: {child.state.move}, Visits: {child.visits}, Wins: {child.wins}')

Move: (0, 1), Visits: 3, Wins: 2
Move: (0, 3), Visits: 7, Wins: 1
Move: (1, 0), Visits: 3, Wins: 2
Move: (1, 1), Visits: 4, Wins: 2
Move: (1, 3), Visits: 3, Wins: 2
Move: (2, 0), Visits: 4, Wins: 2
Move: (2, 1), Visits: 3, Wins: 3
Move: (2, 3), Visits: 3, Wins: 2
Move: (3, 0), Visits: 3, Wins: 2
Move: (3, 1), Visits: 2, Wins: 2
